In [2]:
# import and compile NIH attendees
# read in known applicant files, dedupe and try to merge with applicants file
from collections import Counter
import difflib
import uuid
import itertools
import pandas as pd
import numpy as np
import string
import funcy
import re
import os

In [17]:
APP_DATA_DIR = '~/Dropbox (MIT)/yellowberets/lindsey/intermediate_data/applicants_data'
ATT_DATA_DIR = '~/Dropbox (MIT)/yellowberets/lindsey/intermediate_data/attendees_data'
RAW_ATT_DATA_DIR = '~/Dropbox (MIT)/yellowberets/lindsey/intermediate_data/attendees_data/raw_NIH_data'
CARD_DATA_DIR = '~/Dropbox (MIT)/yellowberets/lindsey/intermediate_data/applicants_data/raw_card_data'
SUM_STAT_DIR = '~/Dropbox (MIT)/yellowberets/lindsey/intermediate_data/summary stats/raw'

In [15]:
r0_file = '1964-1973 associates.XLS'
r2_file = 'Associates alpha by institute.XLS'
r3_file = 'Associates data.XLS'
r4_file = 'NIMH Associates Complete.XLS'
r5_file = 'NINDB Associates alpha by year.xls'
filenames = [r1_file, r2_file, r3_file, r4_file, r5_file]

file_df = map(lambda x: pd.read_excel(os.path.join(RAW_ATT_DATA_DIR, x)), filenames)

file_4_columns = [
    'dno', 'source', 'unknown', 'lastname', 'first_middle', 'institute', 'lab_brch', 
    'program', 'supervisor', 'eod_year', 'med_school', 'year_grad', 'intern_hos', 'intern_dte',
       'res_hosp', 'residency', 'res_dtes'
]

file_df[2].rename(columns={'lname':'lastname', 'fname': 'first_middle'}, inplace=True)
file_df[1].rename(columns={'lname':'lastname', 'fname': 'first_middle'}, inplace=True)

file_df[4].columns = file_4_columns
# for each files in the list, add a column to track source
for name, f in zip(filenames, file_df):
    f.loc[:, 'data_source'] = name
concat_df = pd.concat(file_df)

print sum(map(lambda x: x.shape[0], file_df)) == concat_df.shape[0]

# now we have all the associates, sep first middle into first and middle name, then sort and check 
# to see if we have any duplicates
df2 = pd.concat([concat_df, concat_df.loc[:, 'first_middle'].apply(strip_first_middle)], axis=1)
# consolidate firstname columns
df2.loc[~pd.isnull(df2.first_middle), 'firstname'] = df2.loc[~pd.isnull(df2.first_middle), 'firstname2']
df2.loc[pd.isnull(df2.middlename), 'middlename'] = df2.loc[pd.isnull(df2.middlename), 'middlename2']

df3 = df2.drop(['first_middle', 'firstname2', 'middlename2'], axis=1)

# dropnow where both first and last name are missing
df3 = df3.dropna(subset=['firstname', 'lastname'], how='all')

# df3.dropna(subset=['firstname', 'lastname'], how='all').loc[:, ['firstname', 'lastname', 'dno', 'data_source']]
df3.dropna(subset=['firstname', 'lastname'], how='all').loc[:, 'data_source'].unique()

df3_sorted = df3.sort_values(by=['dno'])

df3_unique = df3.drop_duplicates('dno')

df3_unique.loc[:, 'clean_firstname'] = df3_unique['firstname'].apply(clean_names)
df3_unique.loc[:, 'clean_middlename'] = df3_unique['middlename'].apply(clean_names)
df3_unique.loc[:, 'clean_lastname'] = df3_unique['lastname'].apply(clean_names)

df3_unique.sort_values(['clean_firstname', 'clean_middlename', 'clean_lastname'], inplace=True)

df3_unique.loc[df3_unique.duplicated(['clean_firstname', 'clean_middlename', 'clean_lastname'], keep=False), :].to_csv(
os.path.join(CARD_DATA_DIR, 'attendees_appearing_twice.csv'))

df4 = df3_unique.drop_duplicates(['clean_firstname', 'clean_middlename', 'clean_lastname'], keep='first')

# it seems that dno does refer to unique person, so drop dups based on that 
# save this unique to pick
df3_unique.to_pickle(os.path.join(ATT_DATA_DIR, 'unique_attendees.p'))

# to csv
df3_unique.to_csv(os.path.join(ATT_DATA_DIR, 'unique_attendees.csv'))